# Decision Tree Lab

In [3]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import arff
import math as m

## 1. (40%) Correctly implement the ID3 decision tree algorithm, including the ability to handle unknown attributes (You do not need to handle real valued attributes).  
### Code Requirements/Notes:
- Use standard information gain as your basic attribute evaluation metric.  (Note that normal ID3 would usually augment information gain with gain ratio or some other mechanism to penalize statistically insignificant attribute splits. Otherwise, even with approaches like pruning below, the SSE type of overfit could still hurt us.) 
- You are welcome to create other classes and/or functions in addition to the ones provided below. (e.g. If you build out a tree structure, you might create a node class).
- It is a good idea to use a simple data set (like the lenses data or the pizza homework), which you can check by hand, to test your algorithm to make sure that it is working correctly. 

In [4]:
class Node:
    def __init__(self) -> None:
        self.name = None
        self.childs = {}

In [5]:
class DTClassifier(BaseEstimator,ClassifierMixin):

    def __init__(self,counts=None):
        self.node = Node()  
        self.infoGain = []      

    def fit(self, X, y, feature_names=None):
        self.X = X
        self.y = y
        fType = type(feature_names)
        self.feature_names = ['feature_'+str(i) for i in range(X.shape[1])] if not (fType==list or fType==np.ndarray) else list(feature_names)
        self.id3()

        return self

    def find_information_gain(self, X_ids):
        y = [self.y[y] for y in X_ids]
        labelCategories = list(set(y))
        labelCategoriesCount = [list(y).count(x) for x in labelCategories]
        labelsCount = len(list(self.y))
        informationGain = 0
        for value in labelCategoriesCount:
            informationGain -= (value/labelsCount)*m.log(value/labelsCount,2)
        return informationGain

    def find_entropy_of_a_feature(self, X_ids, feature_id):
        X = [self.X[x][feature_id] for x in X_ids ]
        y = [self.y[y] for y in X_ids]
        xLabelList = list(set(X))
        xLabelsCount = [list(X).count(x) for x in xLabelList]
        instanceCount = len(list(X)) 
        labelEntropyList = []
        for label in xLabelList:
            labelEntropy = 0
            labelIndexes = [i for i in range(len(X)) if X[i]==label ]        
            yNew = [list(y)[i] for i in labelIndexes]
            yNewLabels = list(set(yNew))
            yNewLabelsCount = [list(yNew).count(x) for x in yNewLabels]
            yNewCount = len(yNew)
            for value in yNewLabelsCount:
                labelEntropy -= (value/yNewCount)*m.log(value/yNewCount,2)
            labelEntropyList.append(labelEntropy)

        featureEntropy = sum([xCount*entropy/instanceCount for xCount, entropy in zip(xLabelsCount, labelEntropyList)])
        return featureEntropy

    def find_max_information_gain_feature(self, X_ids, feature_ids):
        infoGain = self.find_information_gain(X_ids)
        maxInfoGain = -1e10
        maxInfoGainFeature = -1
        for id_ in feature_ids:
            entropy = self.find_entropy_of_a_feature(X_ids,id_)
            featureInfoGain = infoGain - entropy
            # print("id"+str(id_)+" "+str(featureInfoGain))
            if featureInfoGain > maxInfoGain:
                maxInfoGain = featureInfoGain
                maxInfoGainFeature = id_
        self.infoGain.append(maxInfoGain)
        return maxInfoGainFeature, self.feature_names[maxInfoGainFeature]

    def id3(self):
        xIds = [x for x in range(self.X.shape[0])]
        featureIds = [x for x in range(self.X.shape[1])]
        self.node = self.id3_recursive(xIds, featureIds, self.node)

    def id3_recursive(self, x_ids, feature_ids, node):
        if not node:
            node = Node()
        labels_in_features = [self.y[x] for x in x_ids]
        if len(set(labels_in_features)) == 1:           
            return  self.y[x_ids[0]]

        if len(feature_ids) == 0:
            return max(set(labels_in_features), key=labels_in_features.count)             
        
        best_feature_id, best_feature_name  = self.find_max_information_gain_feature(x_ids, feature_ids)
        node.name = best_feature_name
        feature_values = list(set([self.X[x][best_feature_id] for x in x_ids]))
        for value in feature_values:
            x_value_ids = [x for x in x_ids if self.X[x][best_feature_id] == value ]
            value_feature_ids = list(feature_ids)
            to_remove = value_feature_ids.index(best_feature_id)
            value_feature_ids.pop(to_remove)
            node.childs[value] = self.id3_recursive(x_value_ids,value_feature_ids, node=None)
        return node

    def predict(self, x:np.array):
        currentNode = self.node
        while isinstance(currentNode, Node):
            nodeName = currentNode.name
            featureIndex = self.feature_names.index(nodeName)
            featureValue = x[featureIndex]
            if featureValue in currentNode.childs:
                currentNode = currentNode.childs[featureValue]
            else:
                return 0
        return currentNode


    def score(self, X, y):
        count = 0
        for x,y_ in zip(X,y):
            yHat = self.predict(x)
#             print('yHat: '+str(yHat)+' y: '+str(y_))
            if yHat == y_:
                count += 1
        return count/len(y)

## 1.1 Debug

Debug your model by training on the lenses dataset: [Debug Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses.arff)

Test your model on the lenses test set: [Debug Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/lenses_test.arff)

Parameters:
(optional) counts = [3,2,2,2] (You should compute this when you read in the data, before fitting)

---

Expected Results: Accuracy = [0.33]

Predictions should match this file: [Lenses Predictions](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/pred_lenses.csv)

*NOTE: The [Lenses Prediction](https://raw.githubusercontent.com/cs472ta/CS472/master/debug_solutions/pred_lenses.csv) uses the following encoding: soft=2, hard=0, none=1. If your encoding is different, then your output will be different, but not necessarily incorrect.*

Split Information Gains (These do not need to be in this exact order):

[0.5487949406953987, 0.7704260414863775, 0.3166890883150208, 1.0, 0.4591479170272447, 0.9182958340544894]

<!-- You should be able to get about 68% (61%-82%) predictive accuracy on the lenses data -->

Here's what your decision tree splits should look like, and the corresponding child node predictions:

Decision Tree:
<pre>
feature_3 = 0:
	feature_2 = 0:
		feature_0 = 0:
			prediction: 2
		feature_0 = 1:
			feature_1 = 0:
				prediction: 2
			feature_1 = 1:
				prediction: 1
		feature_0 = 2:
			prediction: 2
	feature_2 = 1:
		feature_1 = 0:
			feature_0 = 0:
				prediction: 1
			feature_0 = 1:
				prediction: 1
			feature_0 = 2:
				prediction: 0
		feature_1 = 1:
			prediction: 0
feature_3 = 1:
	prediction: 1
</pre>

In [6]:
# Load debug training data 

data_train = arff.loadarff('debug_train.arff')
df_train = pd.DataFrame(data_train[0])
X_train = df_train.iloc[:,:-1].to_numpy().astype(str)
y_train = df_train.iloc[:,-1].to_numpy().astype(str)

# Train Decision Tree

id3 = DTClassifier()
self_ = id3.fit(X_train, y_train)

# Load debug test data

data_test = arff.loadarff('lenses_test.arff')
df_test = pd.DataFrame(data_test[0])
X_test = df_test.iloc[:,:-1].to_numpy().astype(str)
y_test = df_test.iloc[:,-1].to_numpy().astype(str)

# Predict and compute model accuracy

score = self_.score(X_test,y_test)
print('####################')
print('Accuracy: '+str(score))                                         
                                              

# Print the information gain of every split you make.
print('####################')
print("info gain: ")
print(self_.infoGain)
                                              

####################
Accuracy: 0.3333333333333333
####################
info gain: 
[0.5487949406953986, 0.4931334568174778, 0.32917227207875527, 0.3820802083934297, 0.2704260414863775, 0.4897869792568112]


In [7]:
# Optional/Additional Debugging Dataset - Pizza Homework
# pizza_dataset = np.array([[1,2,0],[0,0,0],[0,1,1],[1,1,1],[1,0,0],[1,0,1],[0,2,1],[1,0,0],[0,2,0]])
# pizza_labels = np.array([2,0,1,2,1,2,1,1,0])

## 1.2 Evaluation

We will evaluate your model based on its performance on the zoo dataset. 

Train your model using this dataset: [Evaluation Train Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo.arff)

Test your model on this dataset: [Evaluation Test Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/zoo_test.arff)

Parameters:
(optional) counts = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2] (You should compute this when you read in the data, before fitting)

---
Print out your accuracy on the evaluation test dataset.

Print out the information gain of every split you make.

In [8]:
# Load evaluation training data

data_train = arff.loadarff('zoo.arff')
df_train = pd.DataFrame(data_train[0])
X_train = df_train.iloc[:,:-1].to_numpy().astype(str)
y_train = df_train.iloc[:,-1].to_numpy().astype(str)

# Train Decision Tree

id3 = DTClassifier()
self_ = id3.fit(X_train, y_train)

# Load evaluation test data
data_test = arff.loadarff('zoo_test.arff')
df_test = pd.DataFrame(data_test[0])
X_test = df_test.iloc[:,:-1].to_numpy().astype(str)
y_test = df_test.iloc[:,-1].to_numpy().astype(str)

# Print out the information gain for every split you make

score = self_.score(X_test,y_test)
print('####################')
print('Accuracy: '+str(score))                                         
                                              

# Print the information gain of every split you make.
print('####################')
print("info gain: ")
print(self_.infoGain)



####################
Accuracy: 0.147
####################
info gain: 
[1.3630469031539394, 0.08239443349700448, 0.5313938602577332, 0.3351741869019712, 0.6314784256287459, -0.1532126241347666, 0.25040650904711853, 0.40180311710413813, 0.7295214225955247]


## 2. (20%) You will use your ID3 algorithm to induce decision trees for the cars dataset and the voting dataset.  Do not use a stopping criteria, but induce the tree as far as it can go (until classes are pure or there are no more data or attributes to split on).  
- Implement and use 10-fold Cross Validation (CV) on each data set to predict how well the models will do on novel data.  
- For each dataset, report the training and test classification accuracy for each fold and the average test accuracy. 
- As a rough sanity check, typical decision tree accuracies for these data sets are: Cars: .90-.95, Vote: .92-.95.

## 2.1 Implement 10-fold Cross Validation

In [16]:
# Write a function that implements 10-fold cross validation
from random import randrange

def cross_validation_split(dataset, folds=10):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / folds)
	for i in range(folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split


# print(data_splits[0])
def calculate_cross_validation_accuracy(data_splits):
    accuracyList = []
    for i in range(len(data_splits)):
        test_data = np.array(data_splits[i])    
        train_data = []
        for j in range(len(data_splits)):
            if not j==i:
                for data_point in data_splits[j]:
                    train_data.append(data_point)           
        train_data = np.array(train_data)
        id3 = DTClassifier()
        accuracy = id3.fit(train_data[:,:-1],train_data[:,-1]).score(test_data[:,:-1], test_data[:,-1])
        print("Accuracy_"+str(i)+" : "+str(accuracy))
        accuracyList.append(accuracy)
    return sum(accuracyList)/len(accuracyList)

##  2.2 Cars Dataset
- Use this [Cars Dataset](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/cars.arff)
- Make a table for your K-Fold cross validation accuracies

*If you are having trouble using scipy's loadarff function (scipy.io.arff.loadarff), try:*

*pip install arff &nbsp;&nbsp;&nbsp;&nbsp;          # Install arff library*

*import arff as arf*                   

*cars = list(arf.load('cars.arff'))   &nbsp;&nbsp;&nbsp;&nbsp;# Load your downloaded dataset (!curl, etc.)*

*df = pd.DataFrame(cars)*  

*There may be additional cleaning needed*

In [20]:
# Use 10-fold CV on Cars Dataset
# Report Training and Test Classification Accuracies
# Report Average Test Accuracy

data_ = list(arf.load('cars.arff'))
data_ = np.asarray(data_, dtype=str)
df = pd.DataFrame(data_)
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)
data = df.to_numpy().astype(str)

data_splits = cross_validation_split(data)
accuracy = calculate_cross_validation_accuracy(data_splits)
print('###############')
print("Average Accuracy: "+str(accuracy))

Accuracy_0 : 0.9244186046511628
Accuracy_1 : 0.9186046511627907
Accuracy_2 : 0.8837209302325582
Accuracy_3 : 0.936046511627907
Accuracy_4 : 0.877906976744186
Accuracy_5 : 0.872093023255814
Accuracy_6 : 0.8837209302325582
Accuracy_7 : 0.872093023255814
Accuracy_8 : 0.8430232558139535
Accuracy_9 : 0.9069767441860465
###############
Average Accuracy: 0.8918604651162789


## 2.3 Voting Dataset
- Use this [Voting Dataset with missing values](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)
- Note that you will need to support unknown attributes in the voting data set. 

In [17]:
# Used 10-fold CV on Voting Dataset
# Report Training and Test Classification Accuracies
# Report Average Test Accuracy

data_ = arff.loadarff('voting_with_missing.arff')
df = pd.DataFrame(data_[0])
features = df.columns.values
data = df.to_numpy().astype(str)
df = pd.DataFrame(data, columns=features)
df = df.replace('?', np.nan)
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

data_splits = cross_validation_split(data)
accuracy = calculate_cross_validation_accuracy(data_splits)
print('###############')
print("Average Accuracy: "+str(accuracy))

Accuracy_0 : 0.8837209302325582
Accuracy_1 : 0.9302325581395349
Accuracy_2 : 0.9069767441860465
Accuracy_3 : 0.9534883720930233
Accuracy_4 : 0.9534883720930233
Accuracy_5 : 0.8837209302325582
Accuracy_6 : 0.9069767441860465
Accuracy_7 : 1.0
Accuracy_8 : 0.9767441860465116
Accuracy_9 : 0.9069767441860465
###############
Average Accuracy: 0.9302325581395348


## 2.4 Discuss Your Results

- Summarize your results from both datasets, and discuss what you observed. 
- A fully expanded tree will often get 100% accuracy on the training set. Why does this happen and in what cases might it not?  

Discuss your results

## 3. (15%) For each of the two problems above, summarize in English what the decision tree has learned (i.e. look at the induced tree and describe what rules it has discovered to try to solve each task). 
- If the tree is very large you can just discuss a few of the more shallow attribute combinations and the most important decisions made high in the tree.

## 3.1 Discuss what the decision tree induced on the cars dataset has learned

Discussion Here

## 3.2 Discuss what the decision tree induced on the voting dataset has learned

Discussion Here

## 3.3 How did you handle unknown attributes in the voting problem? Why did you choose this approach? (Do not use the approach of just throwing out data with unknown attributes).

Discuss how you handled unknown attributes

## 4.1 (10%) Use SciKit Learn's decision tree on the voting dataset and compare your results. Try different parameters and report what parameters perform the best on the test set. 

### 4.1.1 SK Learn on Voting Dataset
- Use this [Voting Dataset with missing values](https://raw.githubusercontent.com/cs472ta/CS472/master/datasets/voting_with_missing.arff)

In [12]:
# Use SK Learn's Decision Tree to learn the voting dataset

# Explore different parameters

# Report results

Discuss results & compare to your method's results

## 4.2 (10%) Choose a data set of your choice (not already used in this or previous labs) and use the SK decision tree to learn it. Experiment with different hyper-parameters to try to get the best results possible.

In [13]:
# Use SciKit Learn's Decision Tree on a new dataset

# Experiment with different hyper-parameters

## 5. (5%) Visualize sklearn's decision tree for your chosen data set (using export_graphviz or another tool) and discuss what you find. If your tree is too deep to reasonably fit on one page, show only the first several levels (e.g. top 5).

In [14]:
# Include decision tree visualization here

# Discuss what the model has learned

## 6. (optional 5% extra credit) Implement reduced error pruning to help avoid overfitting.  
- You will need to take a validation set out of your training data to do this, while still having a test set to test your final accuracy. 
- Create a table comparing your decision tree implementation's results on the cars and voting data sets with and without reduced error pruning. 
- This table should compare:
    - a) The # of nodes (including leaf nodes) and tree depth of the final decision trees 
    - b) The generalization (test set) accuracy. (For the unpruned 10-fold CV models, just use their average values in the table).